In [1]:
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [2]:
#pretraitement (mise à echelle (divise chaque pisel/255 bech trodhom f [0,1]))
#définit les transformations à appliquer lorsqu'elle sont chargées(recette)
#trodhom tensors =>normalisation /(najmou namloulhom augmentation)
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)


In [3]:
#lit les imgs , redimensionne,cree la taille du batch , convertir en niv de gris,
#genere des etiquettes pour chaque classe en fct de la structure du repertoire
train_generator = train_data_gen.flow_from_directory(
        'data/train',
        target_size=(48, 48),#les imgs sont redimensionner 48*48
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.


In [4]:
validation_generator = validation_data_gen.flow_from_directory(
        'data/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 7178 images belonging to 7 classes.


In [7]:
from tensorflow.keras.callbacks import LearningRateScheduler
#epoche : nbre d'iteration d'entrainement
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

print(emotion_model.summary())

#cv2.ocl.setUseOpenCL(False)
#learning rate :vitesse à laquelle votre modèle apprend.
#la taille des pas que votre modèle fait pour ajuster ses poids pendant l’entraînement.
# def lr_schedule(epoch):
#      """
#      Learning Rate Schedule
#      """
#      initial_lr = 0.0001
# #     #un facteur de décroissance de 0.1.
#      decay = 0.1
# #     #plus l’époque augmente, plus le taux d’apprentissage diminue.
#      lr = initial_lr * (decay ** epoch)
#      return lr

# # # Define the learning rate scheduler
# lr_scheduler = LearningRateScheduler(lr_schedule)

# Compile the model with Adam optimizer
emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 46, 46, 32)        320       
                                                                 
 conv2d_5 (Conv2D)           (None, 44, 44, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 22, 22, 64)        0         
 g2D)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 22, 22, 64)        0         
                                                                 
 conv2d_6 (Conv2D)           (None, 20, 20, 128)       73856     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 10, 10, 128)       0         
 g2D)                                                 

In [16]:
import scipy as sp

emotion_model_info = emotion_model.fit_generator(
        train_generator,
        #permet de diviser ces données en lots plus petits pour l’entraînement.
        steps_per_epoch=28709 // 64,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=7178 // 64)

C:\Users\asus\AppData\Local\Temp\ipykernel_1412\690059962.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = emotion_model.fit_generator(


Epoch 1/50
448/448 [==============================] - 100s 223ms/step - loss: 0.5419 - accuracy: 0.8009 - val_loss: 1.1694 - val_accuracy: 0.6198
Epoch 2/50
448/448 [==============================] - 87s 194ms/step - loss: 0.5313 - accuracy: 0.8051 - val_loss: 1.1720 - val_accuracy: 0.6143
Epoch 3/50
448/448 [==============================] - 100s 224ms/step - loss: 0.5226 - accuracy: 0.8054 - val_loss: 1.1495 - val_accuracy: 0.6217
Epoch 4/50
448/448 [==============================] - 113s 252ms/step - loss: 0.5123 - accuracy: 0.8125 - val_loss: 1.1595 - val_accuracy: 0.6247
Epoch 5/50
448/448 [==============================] - 118s 264ms/step - loss: 0.5009 - accuracy: 0.8164 - val_loss: 1.1488 - val_accuracy: 0.6200
Epoch 6/50
448/448 [==============================] - 115s 255ms/step - loss: 0.4937 - accuracy: 0.8205 - val_loss: 1.1719 - val_accuracy: 0.6164
Epoch 7/50
448/448 [==============================] - 106s 235ms/step - loss: 0.5005 - accuracy: 0.8177 - val_loss: 1.1749 - 

In [17]:
#architecture
model_json = emotion_model.to_json()
with open("emotion_model.json", "w") as json_file:
    json_file.write(model_json)

In [18]:
#poids
emotion_model.save_weights('emotion_model.h5')